In [1]:
pip install langdetect

     |████████████████████████████████| 981 kB 288 kB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=016dd1043c71a638302cb996693a6b1398240828775565913e1fe7242b35807b
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bpemb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import re
import nltk
import string
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from langdetect import detect
import emoji
from bpemb import BPEmb

In [4]:
df = pd.read_csv("./customer-support-on-twitter/twcs/twcs.csv")
#df = pd.read_csv("../input/customer-support-on-twitter/twcs/twcs.csv")
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
def keepEnglish(d):
    mask = []
    
    for i,doc in enumerate(d["text"]) :
        try:
            if doc and 'en' == detect(doc) :
                mask.append(True)
            else:
                mask.append(False)
        except Exception:
            mask.append(False)

    return mask

In [6]:
mask = keepEnglish(df)
df = df[mask]
df['text'] = df["text"].str.lower()

In [7]:
df_q= df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]
df_qa = pd.merge(df_q, df, left_on='tweet_id', right_on='in_response_to_tweet_id')
qa_tweets = df_qa[["text_x","text_y"]]
qa_tweets.columns = ["questions","answers"]
qa_tweets

,questions,answers
0,@sprintcare is the worst customer service,@115712 can you please send us a private messa...
1,@sprintcare is the worst customer service,@115712 i would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 hello! we never like our customers to ...
3,@115714 y’all lie about your “great” connectio...,@115713 h there! we'd definitely like to work ...
4,"@115714 whenever i contact customer support, t...",@115715 please send me a private message so th...
...,...,...
810149,hai @airasiasupport #asking how many days need...,@823867 we have replied you via dm.thanks-emir
810150,@airasiasupport \n\ni am unable to do web chec...,"@823868 sorry but kindly try to clear browser,..."
810151,@virgintrains hope you are well? does the 9.30...,@524544 that's a peak service. the 09:56 is th...
810152,@115714 wtf!? i’ve been having really shitty s...,"@823869 hey, we'd be happy to look into this f..."


In [8]:
bpemb_en = BPEmb(lang="en", dim=100)

def clean_tweet(tweet):
    lemmatizer = WordNetLemmatizer()
    english_stopwords = set(stopwords.words("english"))
    digit_chr = re.compile("^[a-zA-Z0-9@_!#$%^&*()<>?/\|}{~:,.]*$")
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
   # tokenizer = tokenizers.BertWordPieceTokenizer(strip_accents=False)

    tweets_cleaned = []
    for word in tweet_tokens:
        lemma_word = lemmatizer.lemmatize(word)
        check = digit_chr.match(word)
        if word not in  emoji.UNICODE_EMOJI['en'] and check:
            tweets_cleaned.append(lemma_word)
         
    tweet_clean = " ".join(tweets_cleaned)
    tweet_bpe = bpemb_en.encode(tweet_clean)
    return tweet_bpe

def tweet_counter(df):
    count = Counter()
    for tweet in df.values:
        for word in tweet:
            count[word] += 1
        
    return count


def index_tweet(vocab,tweet_list):
    indice = [vocab[x] for x in tweet_list]
    return indice

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:01<00:00, 391520.88B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 2096413.62B/s]


In [9]:
qa_token = pd.DataFrame(columns = ["question_token", "answer_token"])
qa_token["question_token"] = qa_tweets["questions"].apply(lambda x: clean_tweet(x))
qa_token["answer_token"] = qa_tweets["answers"].apply(lambda x: clean_tweet(x))
qa_token.head()

,question_token,answer_token
0,"[▁is, ▁the, ▁worst, ▁custom, er, ▁service]","[▁can, ▁you, ▁ple, ase, ▁send, ▁u, ▁a, ▁privat..."
1,"[▁is, ▁the, ▁worst, ▁custom, er, ▁service]","[▁i, ▁would, ▁love, ▁the, ▁chance, ▁to, ▁revie..."
2,"[▁is, ▁the, ▁worst, ▁custom, er, ▁service]","[▁hel, lo, ▁, !, ▁we, ▁never, ▁like, ▁our, ▁cu..."
3,"[▁y, ▁all, ▁lie, ▁about, ▁your, ▁great, ▁conne...","[▁h, ▁there, ▁, !, ▁defin, it, ely, ▁like, ▁to..."
4,"[▁whe, never, ▁i, ▁contact, ▁custom, er, ▁supp...","[▁ple, ase, ▁send, ▁me, ▁a, ▁private, ▁message..."


In [10]:
cnt_q = tweet_counter(qa_token["question_token"])
cnt_a = tweet_counter(qa_token["answer_token"])
vocab_ql = sorted(list(cnt_q))
vocab_al = sorted(list(cnt_a))

In [11]:
vocab = set(vocab_ql).union(set(vocab_al))
vocab = sorted(list(vocab))
vocab_dict = {} 
for i, word in enumerate(vocab): 
    vocab_dict[word] = i   
print(len(vocab))

9282


In [12]:
qa_index = pd.DataFrame(columns = ["question_index", "answer_index"])
qa_index["question_index"] = qa_token["question_token"].apply(lambda x: index_tweet(vocab_dict,x))
qa_index["answer_index"] = qa_token["answer_token"].apply(lambda x: index_tweet(vocab_dict,x))
qa_index.head()

,question_index,answer_index
0,"[5943, 8669, 9230, 4360, 822, 8094]","[3736, 9267, 7262, 318, 8067, 8885, 2917, 7404..."
1,"[5943, 8669, 9230, 4360, 822, 8094]","[5716, 9232, 6334, 8669, 3864, 8738, 7827, 866..."
2,"[5943, 8669, 9230, 4360, 822, 8094]","[5573, 1651, 2871, 0, 9116, 6788, 6255, 7007, ..."
3,"[9253, 3106, 6245, 2932, 9272, 5448, 4154, 288...","[5500, 8683, 2871, 0, 4450, 1465, 749, 6255, 8..."
4,"[9149, 1769, 5716, 4196, 4360, 822, 8512, 2879...","[7262, 318, 8067, 6488, 2917, 7404, 6544, 8237..."


In [13]:
qa_index.to_csv("./data/index.csv", index=False)
qa_token.to_csv("./data/token.csv", index=False)
with open('./data/vocab.txt', 'w') as f:
    for item in vocab_dict:
        f.write("%s\n" % item)
